In [1]:
import os
import datetime
import dateutil
import tarfile
import requests
import pandas as pd
import pandas.io.sql as sqlio

%matplotlib inline

In [2]:
'''
amsterdam ne crash plus avec

'''

import glob
def collecte_df(directory_path:str="../boursorama/2020/", market_name:str="amsterdam") ->pd.DataFrame:
    dfglob = glob.glob1(directory_path, market_name + "*")
    files_path = [directory_path + file for file in dfglob]
    df = pd.concat({dateutil.parser.parse(f.split(market_name)[1].split(".bz2")[0]):pd.read_pickle(f) for f in files_path})
    return df



def drop_par_chunks(df, chunk_size=10000):
    def process_chunk(chunk):
        return chunk.dropna()
    
    chunks = []
    for i in range(0, len(df), chunk_size):
        chunks.append(process_chunk(df[i:i + chunk_size]))
    df = pd.concat(chunks)
    return df



def filter(df, year:str = "2020"):
    #amsterdam['last'] = amsterdam['last'].str.replace('(c)', '')
    df["last"] = df["last"].str.replace('(c)', '')
    df["last"] = df["last"].str.replace('(s)', '')
    df["last"] = df["last"].str.replace(' ', '').astype(float)

    df = drop_par_chunks(df)
    df = df.reset_index(level=1, drop=True)
    df = df.rename_axis('date', axis=0)
  
    return df


In [3]:
amsterdam = collecte_df()
amsterdam

last    volume symbol  \
                           symbol                                
2020-08-19 11:41:01.953125 AKTX      1.850(c)    160468   AKTX   
                           FLWS     29.980(c)    426743   FLWS   
                           TXG     105.420(c)    709891    TXG   
                           YI        6.770(c)    655899     YI   
                           PIH       4.505(c)       382    PIH   
...                                       ...       ...    ...   
2020-10-26 12:11:01.817444 CNET      1.510(c)    465775   CNET   
                           ZGEN         9.760         0   ZGEN   
                           ZYNE      3.720(c)    331412   ZYNE   
                           ZYXI     14.470(c)    505732   ZYXI   
                           ZNGA      9.440(c)  15207164   ZNGA   

                                                        name  
                           symbol                             
2020-08-19 11:41:01.953125 AKTX           AKARI THERP SP ADR  
                           FLWS          1-800-FLOWERS.COM-A  
                           TXG             10X GENOMICS RG-A  
                           YI                   111 SP ADS-A  
                           PIH              1347 PROPT INS H  
...                                                      ...  
2020-10-26 12:11:01.817444 CNET               ZW DATA ACTION  
                           ZGEN    ZYMOGENETICS INC  LGT ZLP  
                           ZYNE               ZYNERBA PHARMA  
                           ZYXI                        ZYNEX  
                           ZNGA                   ZYNGA RG-A  

[40738988 rows x 4 columns]

In [4]:
amsterdam = filter(amsterdam)
amsterdam

,last,volume,symbol,name
date,,,,
2020-08-19 11:41:01.953125,1.850,160468,AKTX,AKARI THERP SP ADR
2020-08-19 11:41:01.953125,29.980,426743,FLWS,1-800-FLOWERS.COM-A
2020-08-19 11:41:01.953125,105.420,709891,TXG,10X GENOMICS RG-A
2020-08-19 11:41:01.953125,6.770,655899,YI,111 SP ADS-A
2020-08-19 11:41:01.953125,4.505,382,PIH,1347 PROPT INS H
...,...,...,...,...
2020-10-26 12:11:01.817444,1.510,465775,CNET,ZW DATA ACTION
2020-10-26 12:11:01.817444,9.760,0,ZGEN,ZYMOGENETICS INC LGT ZLP
2020-10-26 12:11:01.817444,3.720,331412,ZYNE,ZYNERBA PHARMA


In [5]:
amsterdam.dtypes

last      float64
volume      int64
symbol     object
name       object
dtype: object